# Neural Network Model

#### to do:

Add VIX and VIX return in the correlation_filter 

manage names in the features: spy_volume, spy_return, spy_close, etc

In [38]:
# Import appropriate modules
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report
import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from vix_functions import garch_fit_and_predict, correlation_filter, retrieve_yahoo_close, retrieve_yahoo_volume 

# Generation of the Features Matrix X

In [39]:
# Ticker List: VIX must be in first position
ticker_list= ["^VIX", "spy", "DX-Y.NYB", "tlt", "ief", "gld", "slv", "CL=F", "USDJPY=X", "FXI", "EZU", "EEM", "EFA"]

In [40]:
# Inclusion of security levels X1
def retrieve_close(close_prices_dict, ticker_list):
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker, start_date='2006-07-02', end_date='2021-10-02')
        close_prices_dict[ticker] = close_price
    return close_prices_dict

close_prices_dict = {}
close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
print("Completed retrieve of close prices")

close_prices_df= pd.DataFrame(close_prices_dict)

close_prices_component_df = correlation_filter(close_prices_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X1=close_prices_component_df
vix=close_prices_df['^VIX']
vix_ret=close_prices_df['^VIX'].pct_change()
VIX=pd.DataFrame([vix, vix_ret]).T
VIX.columns=['VIX','VIX_ret']

X1=pd.concat([VIX,close_prices_component_df], axis=1)

Processing Close ^VIX
Processing Close spy
Processing Close DX-Y.NYB
Processing Close tlt
Processing Close ief
Processing Close gld
Processing Close slv
Processing Close CL=F
Processing Close USDJPY=X
Processing Close FXI
Processing Close EZU
Processing Close EEM
Processing Close EFA
Completed retrieve of close prices


In [41]:
# Inclusion of security returns X2
# Include returns that are correlated more than 0.20 with the Vix return

security_returns_df= close_prices_df.pct_change()
security_returns_component_df = correlation_filter(
                                        security_returns_df, 
                                        min_corr=0.20, 
                                        key_column='^VIX', 
                                        eliminate_first_column=True 
)

X2=security_returns_component_df


#### OBSERVATION: It would be good to include VIX level and Return and check on both in case correlations changes in the future. As of now with some variables I check with VIX level, and others I check VIX return

In [42]:
# inclusion of security volume X3
volume_list = ticker_list[1:len(ticker_list)]

def retrieve_volume(volume_dict, volume_list):
    for ticker in volume_list:        
        volume = retrieve_yahoo_volume(ticker)
        volume_dict[ticker] = volume
    return volume_dict

volume_dict = {}
volume_dict = retrieve_volume(volume_dict, volume_list)
print("Completed retrieve of volume")

volume_df= pd.DataFrame(volume_dict)
volume_df_with_vix=pd.concat([vix, volume_df], axis=1)
#print(volume_df_with_vix.corr())

volume_component_df = correlation_filter(volume_df_with_vix, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )
X3=volume_component_df

Processing Volume spy
Processing Volume DX-Y.NYB
Processing Volume tlt
Processing Volume ief
Processing Volume gld
Processing Volume slv
Processing Volume CL=F
Processing Volume USDJPY=X
Processing Volume FXI
Processing Volume EZU
Processing Volume EEM
Processing Volume EFA
Completed retrieve of volume


In [43]:
# Inclusion of GARCH series X4
garch_series=pd.DataFrame()

for ticker in ticker_list:
        garch_series[ticker]=garch_fit_and_predict(security_returns_df[ticker], ticker, horizon=1, p=1, q=1, o=1)

X4=garch_series
X4


,^VIX,spy,DX-Y.NYB,tlt,ief,gld,slv,CL=F,USDJPY=X,FXI,EZU,EEM,EFA
Date,,,,,,,,,,,,,
2006-07-04,0.667171,0.669192,NaN,0.433795,0.232624,1.332346,2.270001,NaN,0.505921,1.646390,1.064909,2.023401,1.050119
2006-07-05,0.764587,0.715101,NaN,0.460250,0.243572,1.309518,2.233530,NaN,0.496726,1.818623,1.219303,2.268066,1.281258
2006-07-06,0.712915,0.676422,0.427979,0.478023,0.250628,1.295944,2.205288,0.139930,0.485526,1.828120,1.172297,2.184487,1.227976
2006-07-07,0.678683,0.741379,0.427799,0.496775,0.256951,1.274062,2.202950,0.149212,0.620915,1.872258,1.153892,2.209697,1.187973
2006-07-10,0.643754,0.700071,0.436865,0.488067,0.251658,1.259835,2.193299,0.145841,0.595748,1.826972,1.108893,2.114117,1.137894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.905245,0.868685,0.293833,0.866131,0.349633,0.850223,1.530912,0.164903,0.415307,1.935391,1.146791,1.272386,0.967617
2021-09-28,1.505067,1.370846,0.300409,0.902993,0.351506,0.852618,1.510409,0.156697,0.403753,1.863037,1.479471,1.320870,1.299809
2021-09-29,1.334215,1.273171,0.318292,0.878735,0.342399,0.841191,1.642255,0.151903,0.413942,1.811563,1.428190,1.319874,1.243172


In [44]:
# Inclusion of return squares in X5

returns_squared_df_no_vix= security_returns_df.drop(columns='^VIX')**2
returns_squared_and_vix_level_df=pd.concat([vix,returns_squared_df_no_vix], axis=1)
returns_squared_component_df = correlation_filter(returns_squared_and_vix_level_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X5=returns_squared_component_df
X5

,spy,DX-Y.NYB,tlt,ief,gld,slv,FXI,EZU,EEM,EFA
Date,,,,,,,,,,
2006-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-07-04,0.000000,NaN,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2006-07-05,0.000033,NaN,6.374763e-05,0.000013,0.000026,0.000063,0.000698,0.000254,9.639880e-04,3.616540e-04
2006-07-06,0.000008,8.529675e-06,3.753093e-05,0.000010,0.000069,0.000126,0.000715,0.000064,4.173904e-04,1.112225e-04
2006-07-07,0.000042,1.879252e-05,4.157493e-05,0.000010,0.000038,0.000558,0.000355,0.000039,3.597564e-04,2.119929e-05
...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.000008,2.869577e-07,1.351201e-05,0.000007,0.000004,0.000123,0.000207,0.000001,4.223255e-05,3.877590e-07
2021-09-28,0.000406,1.744296e-05,2.426398e-04,0.000015,0.000094,0.000058,0.000002,0.000569,1.667539e-04,4.755389e-04
2021-09-29,0.000003,3.695060e-05,3.010235e-06,0.000001,0.000020,0.001595,0.000052,0.000024,8.679118e-05,5.257838e-06


### Inclusion of Google Trends

In [45]:
# Upload of csv files
keywords=['banking', "consumer", "depression", "gdp", "inflation",
          'unemployment', 'liquidity','cci', 'vix_word']
google_trends_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/Google_trends/{keyword}.csv"
    print(file_path)
    trend=pd.read_csv(Path(file_path),
                      index_col= 'Daily', 
                      parse_dates= True,
                      infer_datetime_format=True)
    google_trends_df=pd.concat([google_trends_df, trend], axis=1)

google_trends_df

./Resources/Google_trends/banking.csv
./Resources/Google_trends/consumer.csv
./Resources/Google_trends/depression.csv
./Resources/Google_trends/gdp.csv
./Resources/Google_trends/inflation.csv
./Resources/Google_trends/unemployment.csv
./Resources/Google_trends/liquidity.csv
./Resources/Google_trends/cci.csv
./Resources/Google_trends/vix_word.csv


,Banking: (United States),Consumer: (United States),Depression: (United States),GDP: (United States),inflation,unemployment: (United States),Liquidity: (United States),cci,VIX:(United States),f
Daily,,,,,,,,,,
2004-01-01,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-02,33,54,81,0.0,43.0,0.0,0.0,0.0,0.0,1.0
2004-01-03,97,22,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-04,27,22,100,0.0,53.0,0.0,0.0,0.0,0.0,1.0
2004-01-05,64,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2021-10-04,78,95,48,23.0,38.0,16.0,16.0,0.0,0.0,1.0
2021-10-05,63,65,26,24.0,78.0,19.0,25.0,0.0,15.0,1.0
2021-10-06,72,87,51,23.0,32.0,12.0,66.0,51.0,29.0,1.0


In [46]:
# We will interpolate with degree 2 in case of missing values
google_trends_df.iloc[0,:]=0

vix_google_trends_df=pd.concat([vix, google_trends_df], axis=1)

#First row set to zero in case there are a lot of missing values
vix_google_trends_df.iloc[0,1:]=0
print(vix_google_trends_df.head(20))

vix_google_trends_df=vix_google_trends_df.interpolate(method="polynomial", order=2)
vix_google_trends_df

            ^VIX  Banking: (United States)  Consumer: (United States)  \
2004-01-01   NaN                         0                          0   
2004-01-02   NaN                        33                         54   
2004-01-03   NaN                        97                         22   
2004-01-04   NaN                        27                         22   
2004-01-05   NaN                        64                          0   
2004-01-06   NaN                        45                         36   
2004-01-07   NaN                        41                         79   
2004-01-08   NaN                        74                         67   
2004-01-09   NaN                        42                         41   
2004-01-10   NaN                        13                          0   
2004-01-11   NaN                        40                         22   
2004-01-12   NaN                        73                         65   
2004-01-13   NaN                        92         

,^VIX,Banking: (United States),Consumer: (United States),Depression: (United States),GDP: (United States),inflation,unemployment: (United States),Liquidity: (United States),cci,VIX:(United States),f
2004-01-01,NaN,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2004-01-02,NaN,33,54,81,0.0,43.0,0.0,0.0,0.0,0.0,1.0
2004-01-03,NaN,97,22,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2004-01-04,NaN,27,22,100,0.0,53.0,0.0,0.0,0.0,0.0,1.0
2004-01-05,NaN,64,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-04,NaN,78,95,48,23.0,38.0,16.0,16.0,0.0,0.0,1.0
2021-10-05,NaN,63,65,26,24.0,78.0,19.0,25.0,0.0,15.0,1.0
2021-10-06,NaN,72,87,51,23.0,32.0,12.0,66.0,51.0,29.0,1.0
2021-10-07,NaN,66,78,51,NaN,NaN,11.0,41.0,NaN,NaN,NaN


In [47]:
# Filtering by correlation X6

google_trends_component_df = correlation_filter(
                                vix_google_trends_df, 
                                min_corr=0.05, 
                                key_column='^VIX', 
                                eliminate_first_column=True)

X6=google_trends_component_df

# We will interpolate so we can fill the missing data only on Google Trends
pro_interpolation_of_X6=pd.concat([vix, X6], axis=1)
pro_interpolation_of_X6=pro_interpolation_of_X6.interpolate(method="polynomial", order=2, axis=0)
pro_interpolation_of_X6
X6 = pro_interpolation_of_X6.iloc[:,1:]
X6

,Banking: (United States),Consumer: (United States),unemployment: (United States),VIX:(United States)
2004-01-01,0,0,0.0,0.0
2004-01-02,33,54,0.0,0.0
2004-01-03,97,22,0.0,0.0
2004-01-04,27,22,0.0,0.0
2004-01-05,64,0,0.0,0.0
...,...,...,...,...
2021-10-04,78,95,16.0,0.0
2021-10-05,63,65,19.0,15.0
2021-10-06,72,87,12.0,29.0
2021-10-07,66,78,11.0,NaN


# GENERATION OF THE FEATURE MATRIX **X**

In [48]:
# Concatenation of all sources of data
XY=pd.concat([X1, X2, X3, X4, X5, X6], axis=1)

# Interpolation is not applied to numerical variables. We are just going to drop those.
XY=XY.dropna()
XY

,VIX,VIX_ret,spy,USDJPY=X,FXI,EZU,EEM,EFA,spy,tlt,...,gld,slv,FXI,EZU,EEM,EFA,Banking: (United States),Consumer: (United States),unemployment: (United States),VIX:(United States)
2007-07-02,15.400000,-0.051140,114.157928,122.230003,31.688372,39.041733,33.705772,53.611881,0.009041,0.005257,...,0.000136,0.000371,0.000338,0.000159,0.000411,2.098385e-04,88,61,21.0,0.0
2007-07-03,14.920000,-0.031169,114.571579,122.470001,32.352470,39.293312,33.974316,53.867043,0.003623,-0.006098,...,0.000019,0.000020,0.000439,0.000042,0.000063,2.265205e-05,76,54,64.0,0.0
2007-07-05,15.480000,0.037533,114.451195,122.949997,32.238976,39.090744,34.124912,53.598789,-0.001051,-0.011206,...,0.000024,0.000029,0.000012,0.000027,0.000020,2.479956e-05,66,57,45.0,0.0
2007-07-06,14.720000,-0.049096,115.052910,123.360001,33.047962,39.515491,34.709667,53.984821,0.005257,-0.004057,...,0.000070,0.000298,0.000630,0.000118,0.000294,5.187247e-05,69,56,22.0,0.0
2007-07-09,15.160000,0.029891,115.143166,123.370003,33.504379,39.688656,35.045971,54.115681,0.000784,0.003594,...,0.000042,0.000000,0.000191,0.000019,0.000094,5.875791e-06,62,51,0.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-24,17.750000,-0.047236,443.910004,110.325996,38.240002,49.959999,50.779999,80.300003,0.001647,-0.009773,...,0.000002,0.000091,0.000441,0.000110,0.000141,8.790382e-05,77,68,12.0,15.0
2021-09-27,18.760000,0.056901,442.640015,110.764000,38.790001,49.900002,51.110001,80.250000,-0.002861,-0.003676,...,0.000004,0.000123,0.000207,0.000001,0.000042,3.877590e-07,64,58,22.0,45.0
2021-09-28,23.250000,0.239339,433.720001,110.963997,38.840000,48.709999,50.450001,78.500000,-0.020152,-0.015577,...,0.000094,0.000058,0.000002,0.000569,0.000167,4.755389e-04,65,77,16.0,44.0
2021-09-29,22.559999,-0.029677,434.450012,111.528000,38.560001,48.470001,49.980000,78.320000,0.001683,0.001735,...,0.000020,0.001595,0.000052,0.000024,0.000087,5.257838e-06,80,44,18.0,44.0


In [49]:
# Set the Signal column
threshold=0.030
vix_ret=vix.pct_change()
XY["Signal"] = 0.0

XY.loc[(XY['VIX_ret'] >= threshold), 'Signal'] = 1

# # Generate the trading signals 1 (entry) or -1 (exit)
# # where 1 is when the ^VIX is greater than 3.6%.
# # where 0 is when the ^VIX  is less than 3.6%.
#for index, row in XY.iterrows():
#    if row["VIX_ret"] >= 0.036:
#        XY.loc[index, "Signal"] = 1.0

# Review the DataFrame
print(XY["Signal"].head())    
XY["Signal"].value_counts()
XY.loc[XY["Signal"]==1, 'VIX_ret']
#XY.shape  ## 3981

2007-07-02    0.0
2007-07-03    0.0
2007-07-05    1.0
2007-07-06    0.0
2007-07-09    0.0
Name: Signal, dtype: float64


2007-07-05    0.037533
2007-07-10    0.158971
2007-07-20    0.112935
2007-07-24    0.103510
2007-07-26    0.145856
                ...   
2021-09-10    0.114362
2021-09-17    0.113430
2021-09-20    0.235464
2021-09-27    0.056901
2021-09-28    0.239339
Name: VIX_ret, Length: 981, dtype: float64

In [50]:
# Figure out the vix returns and filter out returns based on a threshold
vix_ret=vix.pct_change()
vix_ret[vix_ret>=threshold].index
vix_ret.shape


# How many values of the vix we missed due to missing data on other series
compare=pd.concat([XY.loc[XY["Signal"]==1, 'VIX_ret'],vix_ret[vix_ret>=threshold] ], axis=1)
missing_dates=compare.loc[compare["VIX_ret"]!=compare["^VIX"]]
missing_dates=missing_dates.index
missing_dates
vix[missing_dates]

2006-07-05    14.150000
2006-07-12    14.490000
2006-07-13    17.790001
2006-07-17    18.639999
2006-07-20    16.209999
                ...    
2008-08-07    21.150000
2008-08-12    21.170000
2008-08-18    20.980000
2008-08-25    20.969999
2013-10-08    20.340000
Name: ^VIX, Length: 67, dtype: float64

In [51]:
# Define the target set y using the Signal column
y = XY["Signal"]
# Display a sample of y
y

2007-07-02    0.0
2007-07-03    0.0
2007-07-05    1.0
2007-07-06    0.0
2007-07-09    0.0
             ... 
2021-09-24    0.0
2021-09-27    1.0
2021-09-28    1.0
2021-09-29    0.0
2021-09-30    0.0
Name: Signal, Length: 3562, dtype: float64

In [52]:
# Define features set X by selecting all columns but Signal
X = XY.drop(columns=["Signal"]).shift().dropna()


# Review the features DataFrame
X.head()

,VIX,VIX_ret,spy,USDJPY=X,FXI,EZU,EEM,EFA,spy,tlt,...,gld,slv,FXI,EZU,EEM,EFA,Banking: (United States),Consumer: (United States),unemployment: (United States),VIX:(United States)
2007-07-03,15.40,-0.051140,114.157928,122.230003,31.688372,39.041733,33.705772,53.611881,0.009041,0.005257,...,0.000136,0.000371,0.000338,0.000159,0.000411,0.000210,88.0,61.0,21.0,0.0
2007-07-05,14.92,-0.031169,114.571579,122.470001,32.352470,39.293312,33.974316,53.867043,0.003623,-0.006098,...,0.000019,0.000020,0.000439,0.000042,0.000063,0.000023,76.0,54.0,64.0,0.0
2007-07-06,15.48,0.037533,114.451195,122.949997,32.238976,39.090744,34.124912,53.598789,-0.001051,-0.011206,...,0.000024,0.000029,0.000012,0.000027,0.000020,0.000025,66.0,57.0,45.0,0.0
2007-07-09,14.72,-0.049096,115.052910,123.360001,33.047962,39.515491,34.709667,53.984821,0.005257,-0.004057,...,0.000070,0.000298,0.000630,0.000118,0.000294,0.000052,69.0,56.0,22.0,0.0
2007-07-10,15.16,0.029891,115.143166,123.370003,33.504379,39.688656,35.045971,54.115681,0.000784,0.003594,...,0.000042,0.000000,0.000191,0.000019,0.000094,0.000006,62.0,51.0,0.0,38.0


In [53]:
# Check the correlation among various features
X.corr()

,VIX,VIX_ret,spy,USDJPY=X,FXI,EZU,EEM,EFA,spy,tlt,...,gld,slv,FXI,EZU,EEM,EFA,Banking: (United States),Consumer: (United States),unemployment: (United States),VIX:(United States)
VIX,1.000000,0.111272,-0.262490,-0.237066,-0.373336,-0.414108,-0.460732,-0.458616,-0.136986,0.071164,...,0.266625,0.269717,0.433837,0.477338,0.430923,0.471946,-0.233344,-0.195948,0.009851,0.084776
VIX_ret,0.111272,1.000000,0.003205,0.018562,-0.014417,-0.002641,-0.016263,-0.002082,-0.718252,0.333575,...,0.109264,0.091048,0.047914,0.155982,0.057796,0.133482,-0.013367,-0.004881,-0.023049,0.082663
spy,-0.262490,0.003205,1.000000,0.529318,0.745391,0.816832,0.791345,0.924679,0.026095,-0.014195,...,-0.122835,-0.083801,-0.155475,-0.156786,-0.134926,-0.138833,0.001148,0.246018,0.164092,0.201845
USDJPY=X,-0.237066,0.018562,0.529318,1.000000,0.483577,0.627839,0.238444,0.609661,-0.005412,0.002246,...,-0.057449,-0.099238,-0.014199,-0.120442,-0.038059,-0.077602,0.225833,0.031183,-0.028806,0.069709
FXI,-0.373336,-0.014417,0.745391,0.483577,1.000000,0.857584,0.880915,0.848400,0.054191,-0.039252,...,-0.162934,-0.114384,-0.193373,-0.233604,-0.201330,-0.216375,-0.040887,0.135499,0.093140,0.116453
EZU,-0.414108,-0.002641,0.816832,0.627839,0.857584,1.000000,0.847568,0.957495,0.031104,-0.025388,...,-0.130603,-0.111069,-0.144337,-0.233264,-0.162045,-0.200413,0.065909,0.131631,0.028670,0.117046
EEM,-0.460732,-0.016263,0.791345,0.238444,0.880915,0.847568,1.000000,0.879920,0.056775,-0.036638,...,-0.168401,-0.107576,-0.245873,-0.251991,-0.235549,-0.243594,-0.065280,0.223269,0.116234,0.151461
EFA,-0.458616,-0.002082,0.924679,0.609661,0.848400,0.957495,0.879920,1.000000,0.035616,-0.024027,...,-0.152529,-0.124413,-0.194031,-0.240506,-0.191025,-0.215041,0.069501,0.218225,0.088199,0.162125
spy,-0.136986,-0.718252,0.026095,-0.005412,0.054191,0.031104,0.056775,0.035616,1.000000,-0.431050,...,-0.078192,-0.086600,0.082346,-0.044569,0.076636,-0.008122,-0.011087,-0.003945,0.049144,-0.051339
tlt,0.071164,0.333575,-0.014195,0.002246,-0.039252,-0.025388,-0.036638,-0.024027,-0.431050,1.000000,...,0.031129,0.029150,-0.008133,0.072943,0.004004,0.050313,0.002686,0.020498,-0.020407,0.027620


In [54]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=112)

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]

In [55]:
# Create a StandardScaler instance
scaler =  StandardScaler() # MinMaxScaler() #

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [56]:
# Define the the number of inputs (features) to the model
number_input_features = X.shape[1]

# Review the number of features
number_input_features

47

In [57]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [58]:
# Define the number of hidden nodes for the first hidden layer - use the square root of the number of input features times number of output neurons
hidden_nodes_layer1 = int( (number_input_features + number_output_neurons)/2)-3

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


21

In [59]:
# Define the number of hidden nodes for the second hidden layer - hidden layer neurons divided by 2
hidden_nodes_layer2 =  int(np.floor(hidden_nodes_layer1/2))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

10

In [60]:
# Define the number of hidden nodes for the second hidden layer - hidden layer neurons divided by 2
hidden_nodes_layer3 =  int(np.floor(hidden_nodes_layer2/2))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3

5

In [61]:
# Create the Sequential model instance
nn = Sequential()

In [62]:
# Add the first hidden layer
nn.add(Dense
       (
        input_dim=number_input_features, 
           units=hidden_nodes_layer1, 
           activation="relu"
       )
)

In [63]:
# Add the second hidden layer
nn.add(
    Dense(
        units=hidden_nodes_layer2, 
        activation="relu"
    )
)

In [64]:
# Add the third hidden layer
nn.add(
    Dense(
        units=hidden_nodes_layer3, 
        activation="relu"
    )
)

In [65]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(
    Dense(
        units=number_output_neurons, 
        activation="sigmoid"
    )
)

In [66]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 21)                1008      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                220       
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 6         
Total params: 1,289
Trainable params: 1,289
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Compile the Sequential model
nn.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

In [68]:
# Fit the model using 50 epochs and the training data
model_1 = nn.fit(
                X_train_scaled, 
                y_train, 
                epochs=150
)

Epoch 1/150
73/73 [==============================] - 1s 3ms/step - loss: 0.6327 - accuracy: 0.7090
Epoch 2/150
73/73 [==============================] - 0s 4ms/step - loss: 0.6034 - accuracy: 0.7240
Epoch 3/150
73/73 [==============================] - 0s 3ms/step - loss: 0.5930 - accuracy: 0.7240
Epoch 4/150
73/73 [==============================] - 0s 4ms/step - loss: 0.5880 - accuracy: 0.7240
Epoch 5/150
73/73 [==============================] - 0s 3ms/step - loss: 0.5839 - accuracy: 0.7240: 0s - loss: 0.6131 - accuracy
Epoch 6/150
73/73 [==============================] - 0s 3ms/step - loss: 0.5800 - accuracy: 0.7240
Epoch 7/150
73/73 [==============================] - 0s 3ms/step - loss: 0.5770 - accuracy: 0.7240
Epoch 8/150
73/73 [==============================] - 0s 4ms/step - loss: 0.5745 - accuracy: 0.7240
Epoch 9/150
73/73 [==============================] - 0s 3ms/step - loss: 0.5714 - accuracy: 0.7245
Epoch 10/150
73/73 [==============================] - 0s 3ms/step - loss: 0.568

In [69]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 1s - loss: 1.7008 - accuracy: 0.4890
Loss: 1.7007858753204346, Accuracy: 0.4890333116054535


In [70]:
# Predict values in the test sample and check the accuracy of the predictions.
predictions=nn.predict(X_test_scaled)

predictions[predictions<0.5]=0
predictions[predictions>=0.5]=1

print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

         0.0       0.75      0.44      0.56       893
         1.0       0.29      0.62      0.40       338

    accuracy                           0.49      1231
   macro avg       0.52      0.53      0.48      1231
weighted avg       0.63      0.49      0.51      1231



In [71]:
# Predict the train and test data and concatenate and save it in a dataframe
predictions_x_train = nn.predict(X_train_scaled)
predictions_x_test = nn.predict(X_test_scaled)
predictions_train_test = np.concatenate((predictions_x_train, predictions_x_test), axis=0)
predictions_train_test_df = pd.DataFrame(data=predictions_train_test, index=X.index)

In [72]:
# Rename the columns and index
predictions_train_test_df.columns = ["neural network output"]
predictions_train_test_df.index.name = "Date"

In [73]:
# Review the Predictions train test dataframe
predictions_train_test_df

,neural network output
Date,
2007-07-03,0.245054
2007-07-05,0.983156
2007-07-06,0.448786
2007-07-09,0.459072
2007-07-10,0.220845
...,...
2021-09-24,0.049941
2021-09-27,0.987577
2021-09-28,0.979590


In [74]:
# save the predictions in a csv file
predictions_train_test_df.to_csv("neural_network_output.csv", index=True)